In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [2]:
### Function fire_timeline

def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

def get_lt(lt_string = "Lt_CA_Boreal_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets(files))


def fire_timeline(fireID, 
                  lt,
                  year = '2023',
                  path_region="QuebecGlobalNRT_3571" , 
                  check_last = False, 
                  FWI_source = "station" ):
    
    '''
    '''
    
    ## Read in the largefire file of the fireID
    try:
        fr = fv.load_large_fire(fireID, year = year, path_region= path_region) ## Cluster of 2 fires. 
    except Exception as e:
        print("Fire ID cannot be opened:",fireID)
        print(e)
        return(None)
        ## TO DO Filter? 
            ## VIIRS Static source filter?
            ## WUI filter? 
            
    ## Subset lightning by time and space
    fr = fr.to_crs("3571")
    
    ## TO DO: Figure out which CA ecoregion/province the fire is in and subset lighting by that? 
    print("Not yet subseting spatially beyond quebec. Assuming quebec bounding box")
    
    min_threshold = fr.t.astype('datetime64[ns]').min() - timedelta(days = 10)
    possible_lt = lt[lt.t <= fr.t.min()]
    possible_lt = possible_lt[possible_lt.t >= min_threshold]

    oldest_perim = fr[fr.t == fr.t.max()]
    first_perim = fr[fr.t == fr.t.min()]
    first_perim.geometry = first_perim.buffer(750*2) ## Two viirs pixels???
    join_lt = gpd.sjoin(possible_lt, first_perim, predicate = 'within', how = "inner")
    
    if len(join_lt) == 0:
        join_lt["num_candidates"] = 0
        join_lt["num_strikes"] = len(possible_lt)
        join_lt["num_strikes_10_days"] = len(possible_lt)
    else:
        ## Extract "denominator" or the # of trikes from same period
        denominator = possible_lt[possible_lt.t >= join_lt.t_left.min()]
        denominator = denominator[denominator.t <= join_lt.t_left.max()]
        join_lt["num_candidates"] = len(join_lt)
        join_lt["num_strikes"] = len(denominator)
        join_lt["num_strikes_10_days"] = len(possible_lt)
        
    ## Get distance to individuals ignitions
    # fr["perim_rank"] = fr.t.rank()
    # first_geom = fr[fr.perim_rank == 1].geometry
    # first_geom = first_geom.iloc[0]
    # num_starts = len(first_geom.geoms)
    # for i in range(0, num_starts):
    #     join_lt["dist_start_" + str(i)] = join_lt.distance(first_geom.geoms[i].centroid)
    #     print(fr[fr.perim_rank == 1].to_crs("4326").geometry.iloc[0].geoms[i].centroid)
        
    ## Rank candidate by distance
    # range_geoms = list(range(0, num_starts))
    # string = "dist_start_"
    # columns_dists = [string + str(x) for x in range_geoms]
    # top = len(join_lt) * 1 # Top 100%. Could cut to smaller range
    # dist_bool = join_lt[columns_dists].rank() <= top ## NEED a max distance cutoff. 
    # join_lt["candidate"] = dist_bool.any(axis = 1)
    
    ## Get raw VIIRS pixel timing
    date_string = fr.t.astype("datetime64[ns]").max().strftime("%Y%m%d%p")
    print(date_string)
    raw_obs_times = fv.raw_pixel_times(int(fireID), date_string = date_string, path_region = path_region)
    raw_obs_times = raw_obs_times.reset_index()
    
    ## get station data
    if(FWI_source == "station"):
        print("Assuming Single Quebec Station. 718270-99999.")
        st = pd.read_csv("s3://veda-data-store-staging/EIS/other/station-FWI/19900101.NRT/FWI/718270-99999.linear.HourlyFWIFromHourlyInterpContinuous.csv") ## Corrected record from Robert
        st.HH = st.HH.astype("int")
        st.YYYY = st.YYYY.astype("int")
        st.MM = st.MM.astype("int")
        st.DD = st.DD.astype("int")
        st = fv.date_convert(st)
        
        st_rm = st[["time", "TEMP_C", 'RH_PERC', 'VPD_HPA', 'WDSPD_KPH',
       'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC', 'DC', 'BUI', 'ISI',
       'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH', 'OBSMINUTEDIFF_WDSPD',
       'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW', 'OBSMINUTEDIFF_VIS']]
        st_rm = st_rm.rename(columns = {"time":"t"})
        #### Subset station data by time. 
        st_rm = st_rm[st_rm.t >= min_threshold]
        st_rm = st_rm[st_rm.t <= fr.t.max()]
        
    else:
        #print("No other FWI extraction method ready. Sorry. ")
        raise Exception("No other FWI extraction method ready. Sorry. ")
    
    ## Do merging of all dfs 
    foo = join_lt[["InterCloud", "t_left", "lat_left", "lon_left", "current_mag", "error_elps", "num_station"]]
    foo = foo.rename(columns = {"t_left":"t", "lat_left":"lat", "lon_left":"lon"})
    foo.t = foo.t.astype('datetime64[ns]')
    raw_obs_times = raw_obs_times.rename(columns={"count": "viirs_pix_count"}) 
    raw_obs_times.t = raw_obs_times.t.astype("datetime64[ns]")
    merged = foo.merge(raw_obs_times, on = ["t"], how = "outer")
        
    fr_rm = fr.rename(columns = {"lat":"lat_centroid", "lon":"lon_centroid"})
    fr_rm.t = fr_rm.t.astype("datetime64[ns]")
    merged = merged.merge(fr_rm, on = ["t"], how = "outer")
    
    merged = merged.merge(st_rm, on = ["t"], how = "outer")
    merged["fireID"] = fireID
    
    return(merged)
    

def lf_ids(year = None, regnm = 'BOREAL_NRT_3571_DPS'):
    
    diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

    if year == None:
        year = date.today().year

    if diroutdata.startswith("s3://"):
        # Can't use glob for S3. Use s3.ls instead.
        import s3fs
        s3 = s3fs.S3FileSystem(anon=False)
        s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
        fnms = [f for f in s3.ls(s3path)]


    fnms.sort()
    ids = []
    for f in fnms:
        fnm_lts = os.path.basename(f) 
        one_id = fnm_lts[1:-11]
        ids.append(one_id)
    tmp_ids = pd.DataFrame(ids, columns=["ids"])
    tmp_ids = tmp_ids.ids.unique()
    return(tmp_ids)

def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return(unique_list)

def get_listed_ids(quebec_stats):
    newlist = [x.strip('][\n').split(' ') for x in quebec_stats.fireID.unique()]
    newlist = list(chain(*newlist))
    newlist = [x.replace('\n', ' ') for x in newlist]
    newlist = unique(newlist)
    return(newlist)

# def lf_ids(regnm = 'BOREAL_NRT_3571_DPS', year = None):
    
#     diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

#     if year == None:
#         year = date.today().year

#     if diroutdata.startswith("s3://"):
#         # Can't use glob for S3. Use s3.ls instead.
#         import s3fs
#         s3 = s3fs.S3FileSystem(anon=False)
#         s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
#         fnms = [f for f in s3.ls(s3path)]
#     else:
#         fnms = glob(os.path.join(diroutdata, regnm, str(year), "Largefire"))

#     if len(fnms) > 0:
#         #print("yeah")
#         fnms.sort()
#         ids = []
#         for f in fnms:
#             fnm_lts = os.path.basename(f) ## Can't work, no ordering
#             ids.append(fnm_lts[1:-11])

#     ids = unique(ids)
#     return(ids)
    

In [3]:
lt = get_lt() 

In [4]:
lt = gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt.lon, lt.lat), crs=4326) #4674
lt = lt.to_crs("3571")

In [5]:
     

#tmp = fire_timeline('615', lt = lt, path_region="QuebecGlobalNRT_DPS") #QuebecGlobalNRT_3571

In [6]:
# date_range = pd.date_range(start = "2023-05-01 12:00:00", end = "2023-07-01 12:00:00", freq="12H")
# #date_range_format = datetime.strptime(date_rage, 
# date_snap = date_range.strftime("%Y%m%d%p")

In [7]:
## Get IDs. These IDs come from csvs made by old_shared/fire_weather_vis/Lightning_analysis/snap_prov_lightning.ipynb
# by going through the snapshot files, doing a spatial join, and collecting IDs. 

files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats//boreal_snapstats*.csv")

fire_stats = concat_subsets(files)

fire_stats.t.max()

### Subsetting fire stats by largefire record, so don't waste time looking for IDs that we haven't got yet. Wait, not worth it, size a bigger thing anyway.
#fire_stats = fire_stats[fire_stats.t < "2023-07-20 12:00:00"]

#fire_stats = pd.read_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats/boreal_snapstats_20231024.csv")

Timestamp('2023-08-30 12:00:00')

In [8]:
quebec_stats = fire_stats[fire_stats.prov_name_en == "Quebec"]

tmp_list = get_listed_ids(quebec_stats)

In [9]:
ids_lf = lf_ids()

tmp_list = list(set(tmp_list).intersection(ids_lf))
#tmp_list

In [10]:
from datetime import date
str(date.today().strftime("%Y%m%d"))

'20231030'

In [17]:
#ids = ['12641', '12690','10896','9346']

ids = tmp_list

max_t = "maxT" + str(fire_stats.t.max()) + "_"
min_t = "minT" + str(fire_stats.t.min()) + "_"
year = '2023'
path_region= "BOREAL_NRT_3571_DPS" 
check_last = False 
FWI_source = "station" 

fires = pd.DataFrame()
for n,i in enumerate(ids, start = 0):
    try:
        foo = fire_timeline(i, lt = lt, year = year, path_region= path_region, check_last = False, FWI_source = FWI_source)
    except Exception as e:
        print("Error at ID: ",i)
        print(e)
        continue

    fires = pd.concat([fires, foo])
    #print(fires)
        #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
    fires.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/"+"fire_stats_only_718270-99999_" +min_t + max_t + path_region + FWI_source + str(date.today().strftime("%Y%m%d"))+  ".csv")

['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Largefire/F13150_20230711AM']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


Not yet subseting spatially beyond quebec. Assuming quebec bounding box
20230711AM


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:920: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  data = pickle.load(file)


Assuming Single Quebec Station. 718270-99999.
13150
['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Largefire/F8737_20230603AM']


...

Not yet subseting spatially beyond quebec. Assuming quebec bounding box


/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


20230711PM


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:920: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  data = pickle.load(file)


Error at ID:  13124
[Errno 107] Transport endpoint is not connected
[]
Fire ID cannot be opened: 8641
No objects to concatenate
[]
Fire ID cannot be opened: 12329
No objects to concatenate
12329
[]
Fire ID cannot be opened: 8570
No objects to concatenate
[]
Fire ID cannot be opened: 8552
No objects to concatenate
[]
Fire ID cannot be opened: 8643
No objects to concatenate
[]
Fire ID cannot be opened: 9267
No objects to concatenate
[]
Fire ID cannot be opened: 8877
No objects to concatenate
8877
[]
Fire ID cannot be opened: 13135
No objects to concatenate
[]
Fire ID cannot be opened: 8621
No objects to concatenate
[]
Fire ID cannot be opened: 8502
No objects to concatenate
[]
Fire ID cannot be opened: 13906
No objects to concatenate
[]
Fire ID cannot be opened: 8720
No objects to concatenate
8720
[]
Fire ID cannot be opened: 8551
No objects to concatenate
[]
Fire ID cannot be opened: 12375
No objects to concatenate
[]
Fire ID cannot be opened: 12145
No objects to concatenate
[]
Fire ID 

In [ ]:
#fire_timeline(fireID = '12641' , lt = lt, year = year, path_region= path_region , check_last = False, FWI_source = FWI_source)

In [ ]:
# "20230720PM"

# raw_obs_times = fv.raw_pixel_times(int(12641), date_string = "20230720PM", path_region = "BOREAL_NRT_3571_DPS")

In [ ]:
# # import pickle

# fireID = '12641'
# file = open('/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Serialization/20230720PM.pkl', 'rb')

# # dump information to that file
# data = pickle.load(file)

# # close the file
# file.close()

# fireID = int(fireID)

# times = []
# for i in range(0, len(data.fires[fireID].pixels)):
#     #print(i)
#     times.append(data.fires[fireID].pixels[i].datetime)

In [ ]:
#len(data.fires)

In [ ]:
#len(data.fires[fireID].pixels)

In [18]:
fires.fireID.unique()

array(['13150', '8737', '12147'], dtype=object)

In [22]:
new_fire = fires.groupby("fireID")

In [24]:
fires

,InterCloud,t,lat,lon,current_mag,error_elps,num_station,viirs_pix_count,fireID,n_pixels,...,DC,BUI,ISI,FWI,OBSMINUTEDIFF_TEMP,OBSMINUTEDIFF_RH,OBSMINUTEDIFF_WDSPD,ISPRECREPORTED,OBSMINUTEDIFF_SNOW,OBSMINUTEDIFF_VIS
0,0,2023-07-05 19:25:43.391,51.451367,-77.998767,-9857.0,18.0,2.0,NaN,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2023-07-11 06:03:00.000,NaN,NaN,NaN,NaN,NaN,47.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2023-07-11 06:57:00.000,NaN,NaN,NaN,NaN,NaN,40.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2023-07-11 07:46:00.000,NaN,NaN,NaN,NaN,NaN,6.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2023-07-11 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,93.0,...,NaN,NaN,1.203089,7.831284,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,NaN,2023-07-20 07:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,12147,NaN,...,NaN,NaN,8.577999,35.144007,8040.0,8040.0,8040.0,1.0,8040.0,8040.0
693,NaN,2023-07-20 08:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,12147,NaN,...,NaN,NaN,8.576720,35.140551,8100.0,8100.0,8100.0,1.0,8100.0,8100.0
694,NaN,2023-07-20 09:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,12147,NaN,...,NaN,NaN,8.575451,35.137122,0.0,0.0,0.0,1.0,0.0,0.0
695,NaN,2023-07-20 10:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,12147,NaN,...,NaN,NaN,7.831871,33.087604,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
fires.columns

Index(['InterCloud', 't', 'lat', 'lon', 'current_mag', 'error_elps',
       'num_station', 'viirs_pix_count', 'fireID', 'n_pixels', 'n_newpixels',
       'farea', 'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP',
       'geometry', 'lon_centroid', 'lat_centroid', 'TEMP_C', 'RH_PERC',
       'VPD_HPA', 'WDSPD_KPH', 'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC',
       'DC', 'BUI', 'ISI', 'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH',
       'OBSMINUTEDIFF_WDSPD', 'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW',
       'OBSMINUTEDIFF_VIS'],
      dtype='object')

In [27]:
fires.groupby('fireID').t["Inter"]

In [29]:
fires.fireID.unique()

array(['13150', '8737', '12147'], dtype=object)

In [31]:
smol = fires[fires.fireID == '13150']

In [42]:
smol.columns

Index(['InterCloud', 't', 'lat', 'lon', 'current_mag', 'error_elps',
       'num_station', 'viirs_pix_count', 'fireID', 'n_pixels', 'n_newpixels',
       'farea', 'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP',
       'geometry', 'lon_centroid', 'lat_centroid', 'TEMP_C', 'RH_PERC',
       'VPD_HPA', 'WDSPD_KPH', 'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC',
       'DC', 'BUI', 'ISI', 'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH',
       'OBSMINUTEDIFF_WDSPD', 'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW',
       'OBSMINUTEDIFF_VIS'],
      dtype='object')

In [53]:
first_ig = smol[~smol.InterCloud.isna()].t.min()
print(first_ig)
last_ig = smol[~smol.InterCloud.isna()].t.max()
print(last_ig)


first_detection = smol[~smol.viirs_pix_count.isna()].t.min()
print(first_detection)

feds_start = smol[~smol.farea.isna()].t.min()
print(feds_start)

### Growht threshold



smol["pre_fire"] = ((smol.t >= last_ig) & (smol.t <=  first_detection)) #### first_ig better????????????


2023-07-05 19:25:43.391000
2023-07-05 19:25:43.391000
2023-07-11 06:03:00
2023-07-11 00:00:00


/tmp/ipykernel_1952/566559415.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smol["pre_fire"] = ((smol.t >= last_ig) & (smol.t <=  first_detection)) #### first_ig better????????????


In [54]:
smol

,InterCloud,t,lat,lon,current_mag,error_elps,num_station,viirs_pix_count,fireID,n_pixels,...,BUI,ISI,FWI,OBSMINUTEDIFF_TEMP,OBSMINUTEDIFF_RH,OBSMINUTEDIFF_WDSPD,ISPRECREPORTED,OBSMINUTEDIFF_SNOW,OBSMINUTEDIFF_VIS,pre_fire
0,0,2023-07-05 19:25:43.391,51.451367,-77.998767,-9857.0,18.0,2.0,NaN,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,NaN,2023-07-11 06:03:00.000,NaN,NaN,NaN,NaN,NaN,47.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,NaN,2023-07-11 06:57:00.000,NaN,NaN,NaN,NaN,NaN,40.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,NaN,2023-07-11 07:46:00.000,NaN,NaN,NaN,NaN,NaN,6.0,13150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,NaN,2023-07-11 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,93.0,...,NaN,1.203089,7.831284,0.0,0.0,0.0,1.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,2023-07-10 19:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,NaN,...,NaN,1.630008,10.168700,0.0,0.0,0.0,1.0,0.0,0.0,True
241,NaN,2023-07-10 20:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,NaN,...,NaN,2.342908,13.622396,0.0,0.0,0.0,1.0,0.0,0.0,True
242,NaN,2023-07-10 21:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,NaN,...,NaN,1.308682,8.433525,0.0,0.0,0.0,1.0,0.0,0.0,True
243,NaN,2023-07-10 22:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,13150,NaN,...,NaN,1.436047,9.137693,0.0,0.0,0.0,1.0,0.0,0.0,True


In [45]:
fires[~fires.viirs_pix_count.isna()].t.unique()

array(['2023-07-11T06:03:00.000000000', '2023-07-11T06:57:00.000000000',
       '2023-07-11T07:46:00.000000000', '2023-06-03T06:18:00.000000000',
       '2023-06-03T07:09:00.000000000', '2023-06-03T07:58:00.000000000',
       '2023-07-03T05:12:00.000000000', '2023-07-03T06:03:00.000000000',
       '2023-07-03T06:54:00.000000000', '2023-07-03T07:44:00.000000000',
       '2023-07-03T16:43:00.000000000', '2023-07-03T17:37:00.000000000',
       '2023-07-03T18:26:00.000000000', '2023-07-04T05:44:00.000000000',
       '2023-07-04T06:33:00.000000000', '2023-07-04T07:26:00.000000000',
       '2023-07-04T15:35:00.000000000', '2023-07-04T16:26:00.000000000',
       '2023-07-04T17:15:00.000000000', '2023-07-04T18:06:00.000000000',
       '2023-07-05T05:25:00.000000000', '2023-07-05T06:16:00.000000000',
       '2023-07-05T07:05:00.000000000', '2023-07-05T15:18:00.000000000',
       '2023-07-05T16:07:00.000000000', '2023-07-05T16:58:00.000000000',
       '2023-07-06T05:06:00.000000000', '2023-07-06

In [ ]:
# fr = fr.sort_values(by = ['t'])

# fig, ax = plt.subplots()
# ax.fill_between(upper.index, upper.FWI.rolling(1).mean(), lower.FWI.rolling(5).mean(), 
#                 facecolor='grey', 
#                 alpha=0.2,
#                 label= "95th Percentile")
# ax.fill_between(mid_upper.index, mid_upper.FWI.rolling(1).mean(), mid_lower.FWI.rolling(5).mean(), 
#                 facecolor='grey', 
#                 alpha=0.4,
#                 label= "25th Percentile")
# ax.plot(mean_quant.index, mean_quant.FWI.rolling(1).mean(), 
#         color = "black",
#         label= "Historic Mean Per Day")
# ax.plot(st[(st.time >= "2023-05-01")].time.astype('datetime64[ns]'), st[(st.time >= "2023-05-01")].FWI)
# ax.set_ylabel("Fire Weather Index")
# ax.set_title("2023 Fire Weather Index (FWI) for La Grande Rivière, Quebec, Canada (WMO ID 718270)")
# #ax.legend()
# ax2 = ax.twinx()
# ax2.scatter(candidate.t_left.astype("datetime64[ns]"), candidate.candidate, color = "purple")
# ax2.set_yticklabels("")
# #ax2.plot(fire_stats.t, fire_stats.num_active_fires, color = "red", label = "Number of Fires in Quebec")
# #ax2.legend(loc = 0.5)
# ax3 = ax.twinx()
# #ax3.spines.right.set_position(("axes", 1.2))
# ax3.plot(fr.t.astype("datetime64[ns]"), fr.farea, color = "red")
# #ax3.plot(day_strike.index.astype("datetime64[ns]"), day_strike.InterCloud, color = "orange")

# ax4 = ax.twinx()
# ax4.spines.right.set_position(("axes", 1.2))
# ax4.scatter(raw_obs_times.index, raw_obs_times['count'], color = "orange")

# fig.autofmt_xdate()

# # print("First detection: " + str(fr.t.astype("datetime64[ns]").min()) )
# print(candidate.t_left.min())
# print(candidate.t_left.max())